<a href="https://colab.research.google.com/github/SungWoonJo/RichProject/blob/main/RichProject_Code_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 전체 자동화 루틴 코드 - 급등 분석 및 예측, 텔레그램 알림 포함
# 이 코드는 로컬 또는 클라우드 환경에서 실행 가능하며, 매일 아침 6시에 동작하도록 설정할 수 있습니다.

import ccxt
import pandas as pd
import numpy as np
import datetime as dt
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
import telegram
import os

# 1. 업비트 데이터 수집 함수 (1시간 봉)
def fetch_upbit_ohlcv(ticker, since):
    upbit = ccxt.upbit()
    since_ms = int(since.timestamp() * 1000)
    ohlcv = upbit.fetch_ohlcv(ticker, timeframe='1h', since=since_ms)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# 2. 기술적 지표 계산 함수
def add_technical_indicators(df):
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    macd = MACD(df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    bb = BollingerBands(df['close'])
    df['bb_upper'] = bb.bollinger_hband()
    df['bb_lower'] = bb.bollinger_lband()
    df['golden_cross'] = (df['close'].rolling(5).mean() > df['close'].rolling(20).mean()).astype(int)
    return df

# 3. 급등 종목 찾기 (1시간 내 25% 상승)
def detect_surge(df):
    df['return'] = df['close'].pct_change()
    for i in range(1, len(df)):
        if df['close'].iloc[i] >= df['close'].iloc[i-1] * 1.25:
            return True, df.iloc[i]['datetime']
    return False, None

# 4. 예측 후보 탐색 (간단한 유사 패턴 탐색)
def is_similar_pattern(df):
    last = df.iloc[-1]
    return (last['rsi'] < 40 and last['macd'] > last['macd_signal'] and last['close'] < last['bb_upper'])

# 5. 텔레그램 알림 전송 함수
def send_telegram_message(message):
    TOKEN = os.getenv("8151279259:AAEHCbx42TkDaZ2G-Jo5yWm-1jGTkH1INPE")
    CHAT_ID = os.getenv("1601533002")
    bot = telegram.Bot(token=TOKEN)
    bot.send_message(chat_id=CHAT_ID, text=message)

# 6. 전체 루틴 실행 함수
def run_your_full_analysis():
    upbit = ccxt.upbit()
    markets = upbit.load_markets()
    tickers = [symbol for symbol in markets if symbol.startswith('KRW/') and symbol not in ['KRW/BTC', 'KRW/ETH']]
    result = []

    for ticker in tickers:
        try:
            df = fetch_upbit_ohlcv(ticker, dt.datetime.now() - dt.timedelta(days=180))
            df = add_technical_indicators(df)
            surged, surge_time = detect_surge(df)
            if surged:
                df_recent = df[df['datetime'] < surge_time].tail(30)
                if is_similar_pattern(df_recent):
                    result.append((ticker, surge_time))
        except Exception as e:
            continue

    if result:
        msg = '\n'.join([f"급등 후보: {r[0]} (예상 시점: {r[1]})" for r in result])
    else:
        msg = "예측된 급등 종목이 없습니다."

    send_telegram_message("[업비트 급등 예측 결과]\n" + msg)

# 이 코드는 외부에서 run_your_full_analysis()를 호출하면 실행됩니다.
# 작업 스케줄러 또는 클라우드 예약 실행기로 설정 가능
